In [18]:
from xray import load_model, data, params, trainer
import math
import os
from glob import glob

import pandas as pd
# from xray import data, 
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder


In [19]:
load_previous = False  # If True, modify at the top the loading parameters
filename = "xray_df.csv"
img_size = (224, 224)
job_type = "binary"
split = (0.65, 0.175, 0.175)  # Train Val Test
data_filter = 0.45
cnn_geometry = (1024*8,)  # 512, 256)
dropout_layer = False
dropout_rate = 0.4
batch_size = 32
epochs = 30
# learning_rate = 0.0005

print(f"Start building and training CNN for {job_type}.")

print("Set Parameters")

# Load data
path_to_png = params.GCP_IMAGE_BUCKET
df = data.get_data_from_gcp(filename)

print("Loaded Training Data")

# Train multilabel for sick people. Modify if binary class
# df = df[df["Fixed_Labels"] != "No Finding"]

print(f"Total {len(df)} files loaded")

# Small data ELT
df["path"] = df.path.map(
    lambda x: "/".join(x.split("/")[-3:])
)  # Relative paths to file loc
df.path = df.path.map(
    lambda x: os.path.join(params.GCP_IMAGE_BUCKET, x)
)  # Absolute path in GCP
# df["labels"] = df["Fixed_Labels"].map(
#     lambda x: x.split("|"))  # 'cat_col' not working
df["labels"] = df["Enfermo"]

# OneHot Encode multilabel
# mlb = MultiLabelBinarizer().fit(df.labels)
# y = mlb.transform(df.labels).astype("int16")
# y = y.tolist()

# Binary encode binary labels
mlb = LabelEncoder().fit(df.labels)
y = mlb.transform(df.labels).astype("int16")
y = y.tolist()

print("Finished preprocessing")

# Train, val, test split
df_train, df_val, df_test = data.split_df(
    df, "Patient ID", split, total_filter=data_filter
)
df_train = df_train.path.to_list()
df_val = df_val.path.to_list()
df_test = df_test.path.to_list()


Start building and training CNN for binary.
Set Parameters
Loaded Training Data
Total 112104 files loaded
Finished preprocessing


In [23]:
ds_train = data.make_dataset(path_to_png, 32, df_train, y)
ds_val = data.make_dataset(path_to_png, 32, df_val, y)
ds_test = data.make_dataset(path_to_png, 32, df_test, y, test_set=True)


In [16]:

model_dir = 'models/binary/vgg16/'
filename = '2021-10-20_21:00:26.367062'
dest_dir = '../../models/binary/gg16'
model_bin = load_model.load_model_from_gcp(model_dir, filename, dest_dir)

2021-10-21 14:27:21.877129: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-21 14:27:21.877165: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alienware): /proc/driver/nvidia/version does not exist
2021-10-21 14:27:21.878074: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
model_bin.predict(ds_test)

2021-10-21 14:29:09.906481: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 71 of 1000
2021-10-21 14:29:19.903692: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 153 of 1000
2021-10-21 14:29:30.100431: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 242 of 1000
2021-10-21 14:29:40.190505: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 330 of 1000
2021-10-21 14:29:50.249607: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 381 of 1000
2021-10-21 14:30:00.384719: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 464 of 1000
2021-10-21 14:30:09.948411: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle bu

In [ ]:
mobilenet_1.00_224